# `get*info` RPCs

In the first notebook, we introduced the framework. In this notebook, we'll setup the framework as before, and then run some RPCs on the running nodes.

As before, we start by importing utility functions and setting up some variables.

In [ ]:
from utils import *

srcdir = "../bitcoin/src" # Change this to the path for your bitcoind executable

self = ExampleTest()
setup_test(self, srcdir)

We'll also import matplotlib and numpy so we can graph some data:

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import time
from IPython import display

# `getblockchaininfo`

The `TestNode` class maps calls to unknown methods onto RPC method calls. We'll start by calling the `getblockchaininfo` RPC.

In [ ]:
pprint(self.nodes[0].getblockchaininfo())

In [ ]:
# Define a function to plot the blockchain
def graph_blockchainheight():
    fig, ax = plt.subplots()
    ind = np.arange(1, 4)

    # show the figure, but do not block
    #    plt.show(block=False)

    bcis = [self.nodes[i].getblockchaininfo() for i in range(3)]

    blockheights = [bci['blocks'] for bci in bcis]
    n0, n1, n2 = plt.bar(ind, blockheights)
    colors = ['b', 'g', 'r', 'c', 'm', 'y']

    tips = [bci['bestblockhash'] for bci in bcis]
    tips_set = set(tips)
    tip_color ={}
    for i, hash in enumerate(tips_set):
        tip_color[hash] = colors[i]
    
    n0.set_facecolor(tip_color[tips[0]])
    n1.set_facecolor(tip_color[tips[1]])
    n2.set_facecolor(tip_color[tips[2]])
    ax.set_xticks(ind)
    ax.set_xticklabels(['Node0', 'Node1', 'Node2'])
    ax.set_ylim([0, max(blockheights) + 1])
    ax.set_ylabel('Node height')
    ax.set_title('Node blockchain height')

    # ask the canvas to re-draw itself the next time it
    # has a chance.
    # For most of the GUI backends this adds an event to the queue
    # of the GUI frameworks event loop.
    fig.canvas.draw()

# Draw a graph of the blockchain heights
graph_blockchainheight()

In [ ]:
# Generate two blocks on node0. Node0 is connected to node1, so their tips should advance together
self.nodes[0].generate(2)

In [ ]:
graph_blockchainheight()

In [ ]:
# Generate two blocks on node2. Node2 is isolated so the blocks won't propogate
self.nodes[2].generate(2)

In [ ]:
graph_blockchainheight()

In [ ]:
# Connect node1 to node2. Tips shouldn't sync.
connect_nodes(self.nodes[1], 2)

In [ ]:
graph_blockchainheight()

In [ ]:
# Generate one more block on node2. Node0 and node1 should re-org to node2
self.nodes[2].generate(1)

In [ ]:
graph_blockchainheight()

# Shutdown

Now that we've finished, call `shutdown_test()` to shutdown the nodes and clean up resources.

In [ ]:
shutdown_test(self)